# <center> PSC 09 - Algorithmic study and implementation on a quantum computer of the solution of a system of linear equations  </center>

    
<center> AGUIRRE RAMIREZ Leder Yhivert, ESPINA QUILODRAN Nicolas, LIMOUSIN Elouan, POCHART Thomas et RICHOUX Simon,
<br><br>
</center>
<center> Release date : 25/04/2021 </center>
<center> Last modification : 23/12/2021 </center>

## 1. Formal description of the HHL implementation

### 1.1 QLSP Problem <a name="QLSP"></a>
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$
We consider a linear system of equations with N variables and N unknowns that can be written as $Ax = b$, where $x$, $b$ are respectively the solution (unknown) and the constant vector. If the matrix $A$ is also assumed to be invertible, such a solution of this problem is given by $x = A^{-1} b$. In the literature, this problem is called Linear System Problem (LSP).

A quantum version of the same problem is called the Quantum Linear System Problem (QLSP), which can be rigorously defined as follows:

**Definition 1** (QLSP). Let $A$ be a Hermitian $N × N$ matrix of determine equal to $1$. Let $b$ be a non-zero vector of $\mathbb{C}^n$ and $x=A^{-1}b$. We suppose that the vectors b and x can be represented as the following quantum states:

$$ \ket{b} := \frac{\sum_i b_i \ket{i}}{\|\sum_i b_i \ket{i}\|_2}
$$

and

$$ \ket{x} := \frac{\sum_i x_i \ket{i}}{\|\sum_i x_i \ket{i}\|_2}
$$

where the state vectors $\ket{i}$ are the pure states of the qubits of a quantum computer. The goal of the problem is to find, at a fixed error parameter $\epsilon$, a quantum state $\ket{\tilde{x}}$ satisfying the following condition:

$$
\mathbb{P}\left( \| \ket{\tilde{x}} - \ket{x} \|_2 < \epsilon \right) > \frac{1}{2}
$$

**Notes:** 

- From the definition above we have: $A\ket{x} = \ket{b}$. Therefore, $A$ is acting both as a matrix and as an operator. 

- As an additional point, most of the time we will work with the binary representation of the fractional part of $i \in [0,1]$ inside the bra-ket: $\overline{i_{n-1}\cdots i_0}^2 = [2^ni]$, with $i_k \in \{0,1 \}$ for all $k \in \{0,1,...,n-1 \}$, $n \in \mathbb{N}^*$ and $[x]$ is the integer part of $x$. For instance, the binary representation of the fractional part of $1/2$ and $1/4$ in 2 bits ($n=2$) are $\overline{10}^2$ and $\overline{01}^2$, respectively.

- We will also write $\ket{m}_n$ instead of $\ket{m \mod{2^n}}_n$ for all $m \in \mathbb{Z}$ to simplify all equations.

- This implementation of HHL can handle negative eigenvalues using the two's complement binary representation of negative integers : let's take $a \in \mathbb{N}$, then the two's complement $n$-bits of $a$ is equivalent to the binary representation in n-bits of $2^n-a$.

### 1.2 Principles of the HHL algorithm <a name="Principles_HHL"></a>

By spectral theorem, we represent $A$ in the form 

\begin{equation*}
	A = \sum_{j=1}^{N_b} \lambda_j \ket{u_j}\bra{u_j}
\end{equation*}

where the $\lambda_j$ are the eigenvalues of  $A$ and $\ket{u_j}$ the corresponding eigenvectors respectively. We notice that the solution of the QLSP $A \ket{x} = \ket{b} $ can be described explicitly thanks to the decomposition invoked previously:

\begin{equation*}
	\ket{x} \propto \left( \sum_{j=1}^{N_b} \frac{1}{{\lambda_j}} \ket{u_j}\bra{u_j} \right) \ket{b}
\end{equation*}

Moreover, knowing the decomposition of $\ket{b} $ in the spectral base associated with $A$, $\ket{b} = \sum_{j=1}^{N_b} b_j \ket{u_j}$, the preceding equation can be written more simply:

\begin{equation*}
	\ket{x} = \sum_{j=1}^{N_b}  \frac{1}{\lambda_j}  b_j \ket{u_j}. 
\end{equation*}

Therefore, the key to the HHL algorithm will be to successfully estimate the eigenvalues of $A$ and manage to reverse them. Let’s now give an idea of how the HHL algorithm achieves this. The first important remark is that a quantum computer is naturally capable of operating in the spectral base of $A$ since the state of a physical system does not depend on its representation base. Thus, the application of $\ket{b}$ to $A$ naturally gives us

\begin{equation*}
A \ket{b} = \sum_{j=1}^{N_b} \lambda_j b_j \ket{u_j}.
\end{equation*}

However, one cannot generally perform such an operation with a quantum computer since the matrix $A$ is a priori not unitary and cannot therefore be seen as the matrix corresponding to the operation carried out by a certain quantum circuit. Instead of $A$, we use the matrix 

\begin{equation*}
U_t := e^{iAt} = \sum_{j=1}^{N_b} e^{i\lambda_j t} \ket{u_j}\bra{u_j}.
\end{equation*}

\noindent where $t\geq 0$. 

The spectrum of $ U_t $ being made up of $e^{i\lambda_j t}$, to ensure that there is a bijection between the spectrum of $U_t$ and that of $A$, $\lambda_j t$ must all be included in the interval $[-\pi, \pi [$, taking $t = \frac{2\pi}{2\cdot \max_j{|\lambda_j|}}$ ensures such a condition. In practice, the value of $t$ have to be tuned since the eigenvalues $\lambda_j$ are unknown a priori and very small values of $t$ will force us to increase the size of the register $n_l$ to estimate accurately the relative phases.

The matrix $U$ being unitary, we can hope to approximate it by a quantum circuit, and an algorithm making it possible to carry out this construction is called an algorithm of **Hamiltonian simulation series** which we will discuss in detail in a dedicated section. 

### 1.3 Steps of the HHL algorithm

For the HHL algorithm we have 3 registers allowing to store the different input parameters of the algorithm (seen as a function). By abuse of notation, we will denote each quantum register by the variable that represents its size:
- The register $ n_l $ contains the phase approximation of the eigenvalues of $ e ^ {iAt} $ after execution of the Quantum Phase Estimation algorithm, the size of this register is a parameter of the algorithm that we will discuss during phases of numerical experiments of the algorithm.

- The register $ n_b $ is used to record the state $ \ket{b} $, in practice if the matrix is of size $ 2 ^ n $, the size of this register will be $ n $. At the end of the execution of the algorithm and if the latter is successful, this same register will contain an approximation of $ \ket{x} $.

- The auxiliary register $ n_S $ of size $ 4n_l + 1 $. The first $ 4n_l $ qubits are used to invert the estimated relative phase after QPE and the last qubit is used as a flag variable for the success of one run of the HHL algorithm. In many articles only the last qubit is considered but in practice this is not the case as we can see in this implementation.

We will note $ n_l = a $ to say that the register $ n_l $ is of size equal to $ a $ and likewise the other registers. Having said this, the steps of the HHL algorithm are:

**1) Loading data**

Initially, the quantum state of the complete system is $\ket{0}_{n_S}\ket{0}_{n_l}\ket{0}_{n_b}$. Since we can standardize the system to be solved, we can assume that $\| b \|=1$. Using the so-called qRAM loading algorithm, the state of the registers is transformed as follows:

\begin{align*}
\text{QRAM Loading: } \ket{0}_{n_S}\ket{0}_{n_l}\ket{0}_{n_b} \mapsto \ket{0}_{n_S}\ket{0}_{n_l}\ket{b}_{n_b} = \sum_j b_j \ket{0}_{n_S}\ket{0}_{n_l}\ket{u_j}_{n_b}
\end{align*}

**2) Quantum Phase Estimation**

Now, notice that the eigenvalues of $U_t$ all have a module equal to 1, so that only the phase of the exponential counts. More precisely, by noting $e^{i\lambda_j t} = e^{i 2 \pi \theta_j}$ for all $j \in \{1, \cdots 2^{n_l} \}$, we want to estimate the relative phases $\theta_j$. To achieve this, we will use the QFT-based Quantum Phase Estimation algorithm:

\begin{align*}
\text{QFT-based QPE: }\sum_j b_j \ket{0}_{n_S}\ket{0}_{n_l}\ket{u_j}_{n_b} \mapsto \sum_j b_j \ket{0}_{n_S}\ket{2^{n_l}\tilde{\theta_j}}_{n_l}\ket{u_j}_{n_b}
\end{align*}

where $\tilde{\theta_j}$ is a supposedly "good" approximation of the fractional part of $\theta_j$ in his binary representation over $n_l$ bits: $\tilde{\theta_j}=[2^{n_l}\theta_j]$.

- If $2^{n_l}\theta_j$ is an integer, then $\ket{2^{n_l}\tilde{\theta_j}}_{n_l}$ is a pure state that contains the exact approximation encoded in the register $n_l$. 
- If on the contrary $2^{n_l}\theta_j$ is not an integer, $\ket{2^{n_l}\tilde{\theta_j}}_{n_l}$ is a mixed state so that the best approximations $\ket{[2^{n_l}\theta_j]}_{n_l}$ are more likely to be measured. It can be shown that:
    - the probability of getting $\ket{\lfloor 2^{n_l}\theta_j \rfloor}_{n_l}$ **or** $\ket{\lceil 2^{n_l}\theta_j \rceil}_{n_l}$ is strictly greater than $8/\pi^2 \approx 0,81$.
    - the probability of getting $\ket{\lfloor 2^{n_l}\theta_j \rfloor+\epsilon}_{n_l}$ is bounded superiorly by $1/|\epsilon|^2$.
    
**Remark:** The construction of the controlled unitary operator $ Controlled-U $, where $U = e ^ {iAt}$, is carried out using the classical Hamiltonian simulation by means of Trotter-Suzuki decompositions: we find a sequence of unitary operators $U_1, ..., U_n$ and a positive integer $ r $ so that $U \approx \tilde {U} = (U1 * ... * Un)^r$. The decomposition is found using a greedy algorithm and coloring approach with a classical time complexity of $\mathcal{O} \left( Ns^2 + \frac{1}{\epsilon_H} \right)$ where N is the number of rows / columns of matrix A, $ s $ its sparsity and $ epsilon_H $ the allowed error in the approximation of $ U $ by $\tilde {U}$.

**3) Inversion of eigenvalues**

The next step will consist in inverting them the estimates of $\theta_j$ and make them appear in the amplitudes of probability of the system. Basically, we would like to use an operator to perform the operation $\ket{2^{n_l}\tilde{\theta_j}}_{n_l} \mapsto \frac{1}{\tilde{\theta_j}} \ket{2^{n_l}\tilde{\theta_j}}_{n_l}$. However, this brings up two problems. The first, of a mathematical nature, is that if $\frac{1}{\tilde{\theta_j}}$ is strictly greater than 1, then this quantity cannot by definition be an amplitude of probability. The second, quantum in nature, is that this operation would not define a unitary operator, since it does not keep the norm of kets. The solution imagined by the designers of the algorithm
HHL is to use the following operator:

\begin{align*}
\text{EIGENROT $\circ$ ASNINV_C: } \sum_j b_j \ket{0}_{S}\ket{0}_{S'}\ket{2^{n_l}\tilde{\theta_j}}_{n_l}\ket{u_j}_{n_b} \mapsto \sum_j b_j \left( \sqrt{1-\frac{C^2}{\tilde{\theta_j}^2}} \ket{0}_{S} + \frac{C}{\tilde{\theta_j}} \ket{1}_{S} \right) \ket{0}_{S'} \ket{2^{n_l}\tilde{\theta_j}}_{n_l}\ket{u_j}_{n_b}
\end{align*}


where $C$ is a normalization constant chosen so that $C/\theta_j$ is less than $1$ for all $j$. In practice, choosing $C = \theta_{min}/2 = t \lambda_{min} / \left(4\pi\right)$ ensures such a condition. In this operator the $\ket{i}_{S}\ket{j}_{S'} = \ket{2i+j}_{n_S}$ for $i,j$ already in binary, so $\ket{0}_{n_S} = \ket{0}_S \ket{0}_{S'}$.

The operator ASNINV correspond to the operation $\ket{2^n_l\tilde{\theta_j}}_{n_l} \ket{2^{n_l}C}_{n_r} \ket{0}_{n_q} \mapsto \ket{2^n_l\tilde{\theta_j}}_{n_l} \ket{r}_{n_r} \ket{q}_{n_q}$ where $2^{n_l}C = 2^n_l\tilde{\theta_j}q+r$ for $r \leq 2^{n_l}p$. In other words, this operator performs the Euclidean division between $C$ and $\tilde{\theta_j}$. Notice that all the sub-registers in this sub-step are of size $n_l$ and they constitute the register $S'$. Afther that, the operator EIGENROT puts the result of the Euclidean division  as an amplitude using the qubit in the register $S$.